In [1]:
 import pandas as pd

In [2]:
df = pd.read_csv('taxi_dataset_with_predictions.csv', index_col=0)

In [3]:
df.head()

,vendor_id,pickup_datetime,passenger_count,store_and_fwd_flag,trip_duration,distance_km,prediction_1,prediction_2
id,,,,,,,,
id2875421,1,2016-03-14 17:24:55,930.399753,0,455.0,1.500479,578.156451,355.270710
id2377394,0,2016-06-12 00:43:35,930.399753,0,663.0,1.807119,962.657188,674.295781
id3858529,1,2016-01-19 11:35:24,930.399753,0,2124.0,6.392080,2546.180515,2422.132431
id3504673,1,2016-04-06 19:32:31,930.399753,0,429.0,1.487155,737.926214,795.992362
id2181028,1,2016-03-26 13:30:55,930.399753,0,435.0,1.189925,666.070794,-4.158492



$$
MSE = \frac{1}{n} \sum_i^n (a(x_i)-y_i)^2
$$


In [4]:
#if we dont use numpy - 1st way using series - fast
error_1 = 0
mse1 = (df["prediction_1"]-df["trip_duration"])**2
error_1 = mse1.sum()
error_1 = error_1/df.shape[0]

In [5]:
#2nd way using loop - slow
error_2 = 0
for index, row in df.iterrows():
    rez = (row["prediction_2"]-row["trip_duration"])**2
    error_2 += rez
error_2 = error_2/df.shape[0]
error_2

124936.14931386459

In [6]:
print(f"MSE первой модели равно: {int(error_1)}")
print(f"MSE второй модели равно: {int(error_2)}")

MSE первой модели равно: 99994
MSE второй модели равно: 124936



$$
RMSE = \sqrt{MSE} = \sqrt{\frac{1}{n} \sum_i^n (a(x_i)-y_i)^2}
$$

In [7]:
error_1 = (error_1)**0.5
error_2 = (error_2)**0.5

In [8]:
print(f"RMSE первой модели равно: {int(error_1)}")
print(f"RMSE второй модели равно: {int(error_2)}")

RMSE первой модели равно: 316
RMSE второй модели равно: 353


$$
MAE = \frac{1}{n} \sum_i^n |a(x_i)-y_i|
$$


In [9]:
mae1 = abs(df["prediction_1"]-df["trip_duration"])
absolute_error_1 = mae1.sum()/df.shape[0]

In [10]:
mae2 = abs(df["prediction_2"]-df["trip_duration"])
absolute_error_2 = mae2.sum()/df.shape[0]

In [11]:
print(f"MAE первой модели равно: {int(absolute_error_1)}")
print(f"MAE второй модели равно: {int(absolute_error_2)}")

MAE первой модели равно: 300
MAE второй модели равно: 281




MAE and MSE for 2 models can give different results (ex. MAE is better for 1st, MSE better for second)
what does it mean?

actually we can make some consiquenses from it

MSE better if number of fatal errors is less
MAE better if average score is better, but the number of fatal errors can be more

In [12]:
# for 1st model difference between predction and real result 
dif1 = df["prediction_1"]-df["trip_duration"]
counter_1 = dif1[abs(dif1)>=500].size
counter_1

33061

In [13]:
dif2 = df["prediction_2"]-df["trip_duration"]
counter_2 = dif2[abs(dif2)>=500].size
counter_2

228789

In [14]:
print(f"Количество отклонений >= 500 от верного ответа для первой модели равно: {counter_1}")
print(f"Количество отклонений >= 500 от верного ответа для второй модели равно: {counter_2}")

Количество отклонений >= 500 от верного ответа для первой модели равно: 33061
Количество отклонений >= 500 от верного ответа для второй модели равно: 228789


MSE, RMSE and MAE - symmetric metrics (=mistakes -2 and +2 are same in this metrics)

sometimes we have to use unsymmetric metrics (-2 and +2 different)

RMSLE - root mean squred log error

$$
\text{RMSLE}(X, y, a) = \sqrt{\frac{1}{\ell}\sum_{i=1}^{\ell} \big(\log{(y_i + 1)} - \log{(a(x_i) + 1)}\big)^2}
$$


Переменные с метриками назовите *rmsle_1*, *rmsle_2*. Округлите их до 3 знаков после точки.

In [15]:
### as we have log - all negative prediction we would turn into 0
df["prediction_1"] = df["prediction_1"].apply(lambda x: max(x, 0))
df["prediction_2"] = df["prediction_2"].apply(lambda x: max(x, 0))

In [16]:
df.head()

,vendor_id,pickup_datetime,passenger_count,store_and_fwd_flag,trip_duration,distance_km,prediction_1,prediction_2
id,,,,,,,,
id2875421,1,2016-03-14 17:24:55,930.399753,0,455.0,1.500479,578.156451,355.270710
id2377394,0,2016-06-12 00:43:35,930.399753,0,663.0,1.807119,962.657188,674.295781
id3858529,1,2016-01-19 11:35:24,930.399753,0,2124.0,6.392080,2546.180515,2422.132431
id3504673,1,2016-04-06 19:32:31,930.399753,0,429.0,1.487155,737.926214,795.992362
id2181028,1,2016-03-26 13:30:55,930.399753,0,435.0,1.189925,666.070794,0.000000


In [17]:
import numpy as np

In [18]:
def rmsleCalculate(prediction, real):
    prediction = np.array(prediction)
    real = np.array(real)
    
    log_real = np.log1p(real)
    log_predict = np.log1p(prediction)

    rmsle = (log_real - log_predict) ** 2
    mean_rmsle = np.mean(rmsle)
    rmsle_all = np.sqrt(mean_rmsle)
    return rmsle_all

In [19]:
rmsle_1 = rmsleCalculate(df["prediction_1"], df["trip_duration"])
rmsle_2 = rmsleCalculate(df["prediction_2"], df["trip_duration"])

In [20]:
###Распечатайте значения для обеих моделей. Используйте эти значения для ответа на задание шага 9.
print(f"RMSLE первой модели равно: {rmsle_1}")
print(f"RMSLE второй модели равно: {rmsle_2}")

RMSLE первой модели равно: 0.5537581774590482
RMSLE второй модели равно: 1.5564340528341787


In [21]:
def over_predicted(predicted, real):
    predicted = np.array(predicted)
    real = np.array(real)
    
    over = 0
    for i in range(predicted.size):
        if predicted[i]>real[i]:
            over += 1
    return over

def under_predicted(predicted, real):
    predicted = np.array(predicted)
    real = np.array(real)
    
    under = 0
    for i in range(predicted.size):
        if real[i]>predicted[i]:
            under += 1
    return under

In [22]:
over_predicted_1 = over_predicted(df["prediction_1"], df["trip_duration"])
under_predicted_1 = under_predicted(df["prediction_1"], df["trip_duration"])

In [23]:
print(f"Предсказания первой модели оказались больше действительных в {over_predicted_1} случаях")
print(f"Предсказания первой модели оказались меньше действительных в {under_predicted_1} случаях")

Предсказания первой модели оказались больше действительных в 1456721 случаях
Предсказания первой модели оказались меньше действительных в 1923 случаях


In [24]:
over_predicted_2 = over_predicted(df["prediction_2"], df["trip_duration"])
under_predicted_2 = under_predicted(df["prediction_2"], df["trip_duration"])

In [25]:
print(f"Предсказания второй модели оказались больше действительных в {over_predicted_2} случаях")
print(f"Предсказания второй модели оказались меньше действительных в {under_predicted_2} случаях")

Предсказания второй модели оказались больше действительных в 811778 случаях
Предсказания второй модели оказались меньше действительных в 646866 случаях


review:
RSMLE cooses 1st model
1st model usually overpredict